In [1]:
#import dependencies
import pandas as pd

#load resource file
purchase_data_file = "Resources/purchase_data.csv"

#Read file and store it in a data frame
purchase_data_df = pd.read_csv(purchase_data_file)

#print first five lines to make sure it worked
purchase_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
#create player data frame

player_data_df = purchase_data_df[["SN","Age","Gender"]]
player_data_df = player_data_df.drop_duplicates(subset = ["SN"])

#create purchase data frame
purchases_df = purchase_data_df[["Purchase ID", "SN", "Age","Item ID", "Item Name", "Price"]]

In [3]:
#calculate total number of players

#calculate length of player data dataframe to get total number of players
total_players = len(player_data_df)

#create summary data frame
total_player_df = pd.DataFrame([{"Total Players": total_players}])

total_player_df


,Total Players
0,576


In [4]:
#calculate number of unique items

unique_items = purchases_df["Item Name"].unique()
number_unique_items = len(unique_items)

#calculate average purchase price

average_price = purchases_df["Price"].mean()

#calculate total number of purchases

total_purchases = purchases_df["Purchase ID"].count()

#calculate total revenue

total_revenue = purchases_df["Price"].sum()

#summary data frame
purchasing_analysis_df = pd.DataFrame([{"Number of Unique Items": number_unique_items,
                                       "Average Price": average_price,
                                       "Number of Purchases": total_purchases,
                                        "Total Revenue": total_revenue
                                       }])

#clean data frame
purchasing_analysis_df["Total Revenue"] = purchasing_analysis_df["Total Revenue"].map("${:.2f}".format)
purchasing_analysis_df["Average Price"]= purchasing_analysis_df["Average Price"].map("${:.2f}".format)

#print data frame
purchasing_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [5]:
##calculate percentage and count of male players

#set gender as index
player_gender_df = player_data_df.set_index("Gender")

#use loc to find total number of male, female, and other players
male_players = len(player_gender_df.loc["Male"])
female_players = len(player_gender_df.loc["Female"])
other_players = len(player_gender_df.loc["Other / Non-Disclosed"])

#use calculation of total players to calculate percent of male, female, and other players
percent_male_players = (male_players) / (total_players)
percent_female_players = (female_players) / (total_players)
percent_other_players = (other_players) / (total_players)

#create summary data frame
gender_demographics_df = pd.DataFrame({"Gender": ["Male", "Female", "Other/Non-Disclosed"],
                                        "Total Count" : [male_players, female_players, other_players],
                                        "Percentage of Players": [percent_male_players, percent_female_players, percent_other_players]
                                       })

gender_demographics_df["Percentage of Players"] = gender_demographics_df["Percentage of Players"].map("{:.2%}".format)

demo_analysis_df = gender_demographics_df.set_index("Gender")

demo_analysis_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other/Non-Disclosed,11,1.91%


In [6]:
## purchasing analysis: broken by gender
#purchase count
#average purchase price
#total purchase value
#average purchase total per person by gender

#set Gender as index of larger data frame
gender_purchase_df = purchase_data_df.set_index("Gender")

#calculate purchase count
male_purchases = len(gender_purchase_df.loc["Male"])
female_purchases = len(gender_purchase_df.loc["Female"])
other_purchases = len(gender_purchase_df.loc["Other / Non-Disclosed"])

#calculate average purchase price
male_avg_price = (gender_purchase_df.loc["Male", "Price"]).mean()
female_avg_price = (gender_purchase_df.loc["Female", "Price"]).mean()
other_avg_price = (gender_purchase_df.loc["Other / Non-Disclosed","Price"]).mean()

#calculate total purchase value
male_purchase_total = (gender_purchase_df.loc["Male", "Price"]).sum()
female_purchase_total = (gender_purchase_df.loc["Female", "Price"]).sum()
other_purchase_total = (gender_purchase_df.loc["Other / Non-Disclosed","Price"]).sum()

#calculate avgerage purchase total per person using total purchases 
avg_purchase_male = (male_purchases) / (total_purchases)
avg_purchase_female = (female_purchases) / (total_purchases)
avg_purchase_other = (other_purchases) / (total_purchases)

#create summary data frame
gender_purchase_analysis_df = pd.DataFrame({"Gender":["Female", "Male", "Other/Non-Disclosed"],
                                "Purchase Count": [female_purchases, male_purchases, other_purchases],
                                "Average Purchase Price": [female_avg_price, male_avg_price, other_avg_price],
                                "Total Purchase Value": [female_purchase_total, male_purchase_total, other_purchase_total],
                                "Avg Total Purchase per Person": [avg_purchase_female, avg_purchase_male, avg_purchase_other]
})

#clean data frame
gender_purchase_analysis_df["Average Purchase Price"] = gender_purchase_analysis_df["Average Purchase Price"].map("${:.2f}".format)
gender_purchase_analysis_df["Total Purchase Value"] = gender_purchase_analysis_df["Total Purchase Value"].map("${:,.2f}".format)
gender_purchase_analysis_df["Avg Total Purchase per Person"] = gender_purchase_analysis_df["Avg Total Purchase per Person"].map("${:.2f}".format)

gender_purchase_analysis_df = gender_purchase_analysis_df.set_index("Gender")
gender_purchase_analysis_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$0.14
Male,652,$3.02,"$1,967.64",$0.84
Other/Non-Disclosed,15,$3.35,$50.19,$0.02


In [7]:
## Purchasing Analysis broken by age using bins

#create bins to hold the ages in
bins = [0, 9, 14, 19, 24, 29, 34, 39, 80]

#name the bin categories
age_bins = "<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"

#categorize existing plyaers using age bins
player_data_df["Age Ranges"] = pd.cut(player_data_df["Age"], bins=bins, labels=age_bins, include_lowest = True)

#create groupby object
age_data_groups = player_data_df.groupby("Age Ranges")

#calculate total count 
age_data = age_data_groups["Age Ranges"].count()

#for percent, take total count colum and divide by number of players
percent_age = age_data/total_players

#create summary data frame
age_range_df = pd.DataFrame({"Total Count":age_data, "Percentage of Players": percent_age})

#clean data frame
age_range_df["Percentage of Players"] = age_range_df["Percentage of Players"].map("{:.2%}".format)

age_range_df

,Total Count,Percentage of Players
Age Ranges,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [8]:
##Purchasing Analysis by Age Demo

#create bins to hold the ages in
bins = [0, 9, 14, 19, 24, 29, 34, 39, 80]

#name the bin categories
age_bins = "<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"

#categorize existing plyaers using age bins
purchases_df["Age Ranges"] = pd.cut(purchases_df["Age"], bins=bins, labels=age_bins, include_lowest = True)

#create grouby object
purchases_grouped = purchases_df.groupby("Age Ranges")

#calculate purchase count
purchase_count = purchases_grouped["Item ID"].count()

#calculate average purchase price
average_purchase_price = purchases_grouped["Price"].mean()

#calculate total purchase value
purchase_value = purchases_grouped["Price"].sum()

#average purchase total per person by age group
purchase_value_total = (purchase_value) / (age_data)

#create summary data frame
age_purchases_df = pd.DataFrame({"Purchase Count": purchase_count, "Average Purchase Price":average_purchase_price, "Total Purchase Value": purchase_value, "Avg Total Purchase per Person": purchase_value_total})

#clean data frame
age_purchases_df["Average Purchase Price"] = age_purchases_df["Average Purchase Price"].map("${:.2f}".format)
age_purchases_df["Total Purchase Value"] = age_purchases_df["Total Purchase Value"].map("${:.2f}".format)
age_purchases_df["Avg Total Purchase per Person"] = age_purchases_df["Avg Total Purchase per Person"].map("${:.2f}".format)

age_purchases_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [65]:
## Identify the top 5 spenders in the game by total purchase count then list
#Purchase Count
#Average Purchase Price
#Total Purchase Value

#use groupby
#we are going to do 3 different groupby, grouping on the "SN" column
#summary stat on the price column
#.sum()
#.mean()
#.count()
#need to be sorted, using the sort_values(), ascending = False

#groupby SN
sn_group = purchases_df.groupby(["SN"])
spenders = sn_group["Price"].sum()

#sort values and find the top 5 spenders
sorted_spenders = spenders.sort_values(ascending=False)
top_spenders = sorted_spenders.iloc[[0,1,2,3,4]]

#calculate purchase count
